# Tensorflow  Layers

In [1]:
%matplotlib inline

In [2]:
from IPython.display import Image
%matplotlib inline

In [3]:
#CellStrat - lets use a high-level tf.layers API to train a multilayer neural network model for
#MNIST classification.

In [4]:
# unzips mnist

import sys
import gzip
import shutil
import os
import numpy as np

In [5]:
import struct
 
def load_mnist(path, kind='train'):
    """Load MNIST data from `path`"""
    labels_path = os.path.join(path, 
                               '%s-labels-idx1-ubyte' % kind)
    images_path = os.path.join(path, 
                               '%s-images-idx3-ubyte' % kind)
        
    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II', 
                                 lbpath.read(8))
        labels = np.fromfile(lbpath, 
                             dtype=np.uint8)

    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols = struct.unpack(">IIII", 
                                               imgpath.read(16))
        images = np.fromfile(imgpath, 
                             dtype=np.uint8).reshape(len(labels), 784)
        images = ((images / 255.) - .5) * 2
 
    return images, labels

In [6]:
## loading the data 
import numpy as np
X_train, y_train = load_mnist('./', kind='train')
print('Rows: %d,  Columns: %d' %(X_train.shape[0], 
                                 X_train.shape[1]))

X_test, y_test = load_mnist('./', kind='t10k')
print('Rows: %d,  Columns: %d' %(X_test.shape[0], 
                                     X_test.shape[1]))
print('Unique classes',(np.unique(y_test)))
## mean centering and normalization:
mean_vals = np.mean(X_train, axis=0)
std_val = np.std(X_train)

X_train_centered = (X_train - mean_vals)/std_val
X_test_centered = (X_test - mean_vals)/std_val

del X_train, X_test

print(X_train_centered.shape, y_train.shape)

print(X_test_centered.shape, y_test.shape)

Rows: 60000,  Columns: 784
Rows: 10000,  Columns: 784
Unique classes [0 1 2 3 4 5 6 7 8 9]
(60000, 784) (60000,)
(10000, 784) (10000,)


In [7]:
# CellStrat - create three fully connected neuron layers using high level tf.layers API
# in the hidden layer, we will use hyperbolic tangent activation functions (tanh),
# and softmax in the output layer

In [8]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

n_features = X_train_centered.shape[1]
n_classes = 10
random_seed = 123
#tf.random.set_seed(random_seed)

g = tf.Graph()
with g.as_default():
    tf.set_random_seed(random_seed)
    tf_x = tf.placeholder(dtype=tf.float32,
                       shape=(None, n_features),
                       name='tf_x')

    tf_y = tf.placeholder(dtype=tf.int32, 
                        shape=None, name='tf_y')
    y_onehot = tf.one_hot(indices=tf_y, depth=n_classes)

    h1 = tf.layers.dense(inputs=tf_x, units=50,
                         activation=tf.tanh,
                         name='layer1')

    h2 = tf.layers.dense(inputs=h1, units=50,
                         activation=tf.tanh,
                         name='layer2')

    logits = tf.layers.dense(inputs=h2, 
                             units=10,
                             activation=None,
                             name='layer3')

    predictions = {
        'classes' : tf.argmax(logits, axis=1, 
                              name='predicted_classes'),
        'probabilities' : tf.nn.softmax(logits, 
                              name='softmax_tensor')
    }

Instructions for updating:
non-resource variables are not supported in the long term


/home/ubuntu/anaconda3/envs/tensorflow/lib/python3.8/site-packages/tensorflow/python/keras/legacy_tf_layers/core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
/home/ubuntu/anaconda3/envs/tensorflow/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1719: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


In [9]:
## define cost function and optimizer:
with g.as_default():
    #cost = tf.losses.softmax_cross_entropy(
    #        onehot_labels=y_onehot, logits=logits) 
    
    cost = tf.losses.softmax_cross_entropy(
            onehot_labels=y_onehot, logits=logits) 
    
    optimizer = tf.train.GradientDescentOptimizer(
            learning_rate=0.1)

    train_op = optimizer.minimize(loss=cost)

    init_op = tf.global_variables_initializer()


In [10]:
def create_batch_generator(X, y, batch_size=128, shuffle=False):
    X_copy = np.array(X)
    y_copy = np.array(y)
    
    if shuffle:
        data = np.column_stack((X_copy, y_copy))
        np.random.shuffle(data)
        X_copy = data[:, :-1]
        y_copy = data[:, -1].astype(int)
    
    for i in range(0, X.shape[0], batch_size):
        yield (X_copy[i:i+batch_size, :], y_copy[i:i+batch_size])

In [11]:
## create a session to launch the graph
sess =  tf.Session(graph=g)
## run the variable initialization operator
sess.run(init_op)

## 50 epochs of training:
training_costs = []
for epoch in range(50):
    training_loss = []
    batch_generator = create_batch_generator(
            X_train_centered, y_train, 
            batch_size=64)
    for batch_X, batch_y in batch_generator:
        ## prepare a dict to feed data to our network:
        feed = {tf_x:batch_X, tf_y:batch_y}
        _, batch_cost = sess.run([train_op, cost],
                                 feed_dict=feed)
        training_costs.append(batch_cost)
    print(' -- Epoch %2d  '
          'Avg. Training Loss: %.4f' % (
              epoch+1, np.mean(training_costs)
    ))


2021-09-21 14:47:43.963345: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-21 14:47:43.964003: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-09-21 14:47:43.967276: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2021-09-21 14:47:43.970549: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:196] None of the MLIR optimization passes are enabled (registered 0 passes)
2021-09-21 14:47:43.992146: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2499995000 Hz


 -- Epoch  1  Avg. Training Loss: 0.3039
 -- Epoch  2  Avg. Training Loss: 0.2327
 -- Epoch  3  Avg. Training Loss: 0.1960
 -- Epoch  4  Avg. Training Loss: 0.1719
 -- Epoch  5  Avg. Training Loss: 0.1543
 -- Epoch  6  Avg. Training Loss: 0.1406
 -- Epoch  7  Avg. Training Loss: 0.1293
 -- Epoch  8  Avg. Training Loss: 0.1197
 -- Epoch  9  Avg. Training Loss: 0.1115
 -- Epoch 10  Avg. Training Loss: 0.1043
 -- Epoch 11  Avg. Training Loss: 0.0979
 -- Epoch 12  Avg. Training Loss: 0.0922
 -- Epoch 13  Avg. Training Loss: 0.0871
 -- Epoch 14  Avg. Training Loss: 0.0825
 -- Epoch 15  Avg. Training Loss: 0.0783
 -- Epoch 16  Avg. Training Loss: 0.0745
 -- Epoch 17  Avg. Training Loss: 0.0710
 -- Epoch 18  Avg. Training Loss: 0.0678
 -- Epoch 19  Avg. Training Loss: 0.0649
 -- Epoch 20  Avg. Training Loss: 0.0622
 -- Epoch 21  Avg. Training Loss: 0.0597
 -- Epoch 22  Avg. Training Loss: 0.0573
 -- Epoch 23  Avg. Training Loss: 0.0552
 -- Epoch 24  Avg. Training Loss: 0.0531
 -- Epoch 25  Av

In [12]:
## do prediction on the test set:
feed = {tf_x : X_test_centered}
y_pred = sess.run(predictions['classes'], 
                  feed_dict=feed)
 
print('Test Accuracy: %.2f%%' % (
      100*np.sum(y_pred == y_test)/y_test.shape[0]))

Test Accuracy: 96.00%
